In [88]:
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [89]:

def obter_dados(fonte, fonte_amigos=None):
    if fonte == 'json':
        with open('cursos.json') as f:
            dados = json.load(f)
    elif fonte == 'webhook':
        url = 'https://exemplo.com/api/dados'
        response = requests.get(url)
        dados = response.json()
    else:
        raise ValueError('Fonte de dados inválida')

    if fonte_amigos == 'json':
        with open('amigos.json') as f:
            amigos = json.load(f)
    elif fonte_amigos == 'webhook':
        url = 'https://exemplo.com/api/amigos'
        response = requests.get(url)
        amigos = response.json()
    else:
        amigos = []

    return dados, amigos

In [91]:
def obter_cursos_avaliados_usuario(usuario_id, df):
    return df[df['id'] == usuario_id]['curso_nome'].unique()

In [92]:
def obter_cursos_avaliados_usuario(usuario_id, df):
    return df[df['id'] == usuario_id]['curso_nome'].unique()

In [93]:
def obter_cursos_avaliados_amigos(usuario_id, df):
    amigos_avaliaram = df[df['amigo_id'].notnull()]['amigo_id'].unique()
    amigos_cursos = []
    for amigo in amigos_avaliaram:
        cursos_avaliados_amigo = df[(df['amigo_id'] == amigo) & (df['usuario_id'] != usuario_id)][['curso_nome', 'avaliacao']]
        if not cursos_avaliados_amigo.empty:
            cursos_avaliados_amigo = cursos_avaliados_amigo.groupby('curso_nome').mean()
            amigos_cursos.append(cursos_avaliados_amigo)
    return pd.concat(amigos_cursos).reset_index()

In [94]:
def recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos):
    df = pd.concat([avaliacoes_usuario, avaliacoes_amigos], ignore_index=True)
    df.drop_duplicates(subset=['curso_nome'], keep='first', inplace=True)
    
    X = df[['curso_media_avaliacao']]
    y = df['curso_nome']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    reg = LinearRegression().fit(X_train, y_train)
    
    cursos_recomendados = list(reg.predict(X_test))
    return cursos_recomendados

In [95]:
fonte = input('Digite a fonte dos dados (json ou webhook): ')
fonte_amigos = input('Digite a fonte dos amigos (json, webhook ou nenhum): ')
dados, amigos = obter_dados(fonte, fonte_amigos)
df = pd.DataFrame(columns=['id', 'nome', 'idade', 'curso_nome', 'curso_categoria', 'curso_avaliacao', 'amigo_id', 'amigo_nome', 'amigo_curso_nome', 'amigo_curso_avaliacao'])

In [96]:
rows = []
for usuario in data['usuarios']:
    for curso in usuario['cursos']:
        row = {
            'id': usuario['id'],
            'nome': usuario['nome'],
            'idade': usuario['idade'],
            'curso_nome': curso['nome'],
            'curso_categoria': curso['categoria'],
            'curso_avaliacao': curso['avaliacao'],
            'amigo_id': None,
            'amigo_nome': None,
            'amigo_curso_nome': None,
            'amigo_curso_avaliacao': None
        }
        rows.append(row)

        for amigo in usuario['amigos']:
            for curso in amigo['cursos']:
                row = {
                    'id': usuario['id'],
                    'nome': usuario['nome'],
                    'idade': usuario['idade'],
                    'curso_nome': None,
                    'curso_categoria': None,
                    'curso_avaliacao': None,
                    'amigo_id': amigo['id'],
                    'amigo_nome': amigo['nome'],
                    'amigo_curso_nome': curso['nome'],
                    'amigo_curso_avaliacao': curso['avaliacao']
                }
                rows.append(row)

df = pd.DataFrame(rows)

In [97]:
df.drop_duplicates(inplace=True)

In [98]:
cursos_agrupados = df.groupby(['curso_nome', 'curso_categoria'])['curso_avaliacao'].mean().reset_index()

In [99]:
cursos_agrupados.rename(columns={'curso_avaliacao': 'curso_media_avaliacao'}, inplace=True)

In [100]:
df = df.merge(cursos_agrupados, on=['curso_nome', 'curso_categoria'], how='left')

In [101]:
df.sort_values(['nome', 'curso_media_avaliacao'], ascending=[True, False], inplace=True)

In [102]:
print(df.head())

   id   nome  idade                                      curso_nome   
0   1  JoÃ£o     25         IntroduÃ§Ã£o Ã  ProgramaÃ§Ã£o em Python  \
9   1  JoÃ£o     25                          InglÃªs IntermediÃ¡rio   
8   1  JoÃ£o     25  Desenvolvimento Web com HTML, CSS e JavaScript   
1   1  JoÃ£o     25                                            None   
2   1  JoÃ£o     25                                            None   

  curso_categoria  curso_avaliacao  amigo_id amigo_nome    amigo_curso_nome   
0   ProgramaÃ§Ã£o              4.5       NaN       None                None  \
9         Idiomas              4.0       NaN       None                None   
8   ProgramaÃ§Ã£o              3.5       NaN       None                None   
1            None              NaN       5.0      Maria  FinanÃ§as Pessoais   
2            None              NaN       5.0      Maria   Marketing Digital   

   amigo_curso_avaliacao  curso_media_avaliacao  
0                    NaN                    4.5 

In [104]:
def recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos):
    avaliacoes = pd.concat([avaliacoes_usuario] + avaliacoes_amigos, axis=0)

    X_train, X_test, y_train, y_test = train_test_split(
        avaliacoes[['curso_media_avaliacao', 'curso_categoria_enc']],
        avaliacoes['curso_avaliacao'],
        test_size=0.2,
        random_state=42
    )
    reg = LinearRegression()
    reg.fit(X_train, y_train)

    cursos_avaliados = avaliacoes_usuario['curso_nome']
    cursos_nao_avaliados = avaliacoes['curso_nome'][~avaliacoes['curso_nome'].isin(cursos_avaliados)]
    cursos_recomendados = []
    for curso in cursos_nao_avaliados:
        curso_avaliacao_media = avaliacoes[avaliacoes['curso_nome'] == curso]['curso_media_avaliacao'].iloc[0]
        curso_categoria_enc = avaliacoes[avaliacoes['curso_nome'] == curso]['curso_categoria_enc'].iloc[0]
        nota_prevista = reg.predict(np.array([[curso_avaliacao_media, curso_categoria_enc]]))[0]
        cursos_recomendados.append((curso, nota_prevista))

    cursos_recomendados = sorted(cursos_recomendados, key=lambda x: x[1], reverse=True)

    return [curso[0] for curso in cursos_recomendados]

In [105]:
avaliacoes_usuario = obter_cursos_avaliados_usuario('usuario1', df)
amigos_avaliaram = df[df['amigo_id'].notnull()]['amigo_id'].unique()
avaliacoes_amigos = obter_cursos_avaliados_amigos('usuario1', df) if len(amigos_avaliaram) > 0 else pd.DataFrame()
cursos_recomendados = recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos)
print(cursos_recomendados)


ValueError: No objects to concatenate